In [1]:
import gym
import math
import random
import numpy as np
from collections import namedtuple
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
env = gym.make('CartPole-v0')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [5]:
state = env.reset()

In [11]:
random.randint(0,1)

0

In [46]:
flag = 0
env.reset()
while True:
    state,reward,done,_ = env.step(random.randint(0,1))
    if done:
        state = None
        flag = 1
    if state is not None:    
        print(state,reward,done)
    if flag==1:
        break
  

[-0.03766803 -0.18497433 -0.00271551  0.31925608] 1.0 False
[-0.04136751 -0.3800575   0.00366962  0.61108141] 1.0 False
[-0.04896866 -0.57523055  0.01589124  0.90491787] 1.0 False
[-0.06047328 -0.38032737  0.0339896   0.61727185] 1.0 False
[-0.06807982 -0.18569632  0.04633504  0.33548491] 1.0 False
[-0.07179375  0.00873663  0.05304474  0.05776613] 1.0 False
[-0.07161902 -0.18710418  0.05420006  0.366702  ] 1.0 False
[-0.0753611  -0.38295275  0.0615341   0.67597048] 1.0 False
[-0.08302016 -0.57887334  0.07505351  0.98737477] 1.0 False
[-0.09459762 -0.77491566  0.094801    1.30265542] 1.0 False
[-0.11009594 -0.58111551  0.12085411  1.04109029] 1.0 False
[-0.12171825 -0.77761739  0.14167592  1.36913787] 1.0 False
[-0.13727059 -0.58452367  0.16905868  1.12391279] 1.0 False
[-0.14896107 -0.78140905  0.19153693  1.46449736] 1.0 False


In [3]:
class ReplayMemory:
    def __init__(self,capacity):
        self.capacity = capacity
        self.memory = []
    
    def push(self,transition):
        if len(self.memory)<self.capacity:
            self.memory.append(transition)
        else:
            self.memory = self.memory[:-1]
            self.memory = [transition] + self.memory
            
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    def memoryLength(self):
        return len(self.memory)

In [4]:
# cartpole game has two discrete actions. Our module which tries to estimate the Q-function takes the state as input
# outputs estimates of Q(state, left) and Q(state, right)
# we use two-layer feed forward network here...(this ofcourse is not the best choice but good enough for our understanding)

class DQN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(DQN, self).__init__()
        
        self.i2h = nn.Linear(input_size,hidden_size)
        self.h2o = nn.Linear(hidden_size,output_size)

    def forward(self, input):
        output = self.i2h(input)
        output = self.h2o(output)
        return output

In [16]:
q_net = DQN(4,25,2)

In [25]:
state = env.reset()
state = torch.from_numpy(state).float().view(1,4)

In [27]:
out = q_net(state)

In [34]:
out.max(1)[1].item()

0

In [32]:
ind.item()

0.07564590871334076

In [35]:
steps_done = 0

GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 10

def select_action(q_net,state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            return q_net(state).max(1)[1].item()
    else:
        return random.randrange(2)

In [37]:
v_net = DQN(4,25,2)
v_net.load_state_dict(q_net.state_dict())

In [ ]:
batch_size = 128
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    

In [ ]:
memory = ReplayMemory(10000)
episode_durationss = []
num_episodes = 1000
for i_episode in range(num_episodes):
    curr_state = env.reset()
    for t in count():
        action = select_action(state)
        next_state, reward, done, _ = env.step(action)
        reward = torch.tensor([reward])
        
        if done:
            next_state = None

        # Store the transition in memory
        c_s_tensor = torch.from_numpy(curr_state).float().view(1,4)
        if next_state is not None:
            n_s_tensor = torch.from_numpy(next_state).float().view(1,4)
        else:
            n_s_tensor = None
        r_tensor = torch.from_numpy(reward).float().view(1,1)
        memory.push(c_s_tensor, action, n_s_tensor, t_tensor)

        # Move to the next state
        curr_state = next_state

        # Perform one step of the optimization (on the target network)
        optimize_model()
        if done:
            episode_durations.append(t + 1)
            break
    # Update the target network
    if i_episode % TARGET_UPDATE == 0:
        v_net.load_state_dict(q_net.state_dict())

print('Complete')
env.close()